In [1]:
import numpy as np
import sys
sys.path.append("../")
import fh_comm as fhc
from scipy.sparse.linalg import norm
max_level = 4

# Test the commutator of basic bosonic operators
The Basic bosonic operators:
$$Ba_i = b^{\dagger}_i+b_i$$
$$Bm_i = b^{\dagger}_i-b_i$$
$$Bn_i = b^{\dagger}_ib_i$$
Theoretically, we could obtain the flowing result:
$$[Ba,Bm] = 2\text{I}$$
$$[Ba,Bn] = -Bm$$
$$[Bm,Bn] = -Ba$$

In [4]:
Ba = fhc.BosonAddOp((0,),1.,max_level)
Bm = fhc.BosonMinuOp((0,),1.,max_level)
Bn = fhc.BosonNumOp((0,),1.,max_level)

print('The result of [Ba,Bm]:')
print(fhc.commutator(Ba,Bm))
print('The result of [Ba,Bn]:')
print(fhc.commutator(Ba,Bn))
print('The result of [Bm,Bn]:')
print(fhc.commutator(Bm,Bn))

The result of [Ba,Bm]:
(2.0) <empty product>
The result of [Ba,Bn]:
(-1.0) Bn_(0,)
The result of [Bm,Bn]:
(-1.0) Bn_(0,)


# compute the result of decomposed hamiltonian 1D

In [2]:
v = -1
u = 1
w0 = 1
g = 1
mod = 1
max_level=4
translatt = fhc.SubLattice(np.array([[2]]))
h0 = fhc.SumOp([fhc.HoppingOp(( 0,), ( 1,), s, v,mod,max_level) for s in [0, 1]])
h1 = fhc.SumOp([fhc.HoppingOp((-1,), ( 0,), s, v,mod,max_level) for s in [0, 1]])
h2 = fhc.SumOp([fhc.ProductOp([fhc.NumberOp((x,), s, 1,mod,max_level) for s in [0, 1]], u) for x in [0, 1]])
h3 = fhc.SumOp([fhc.BosonNumOp((x,), w0, max_level=max_level) for x in [0,1]])
h4 = fhc.SumOp([fhc.ProductOp([fhc.NumberOp((x,),s,1.,mod,max_level), fhc.BosonAddOp((x,),1.,max_level)],g) for s in [0,1] for x in [0,1]])
hlist = [h0,h1,h2,h3,h4]

In [3]:
print('the result of [h4,h2]:')
comm_4_2 = fhc.commutator_translation(h4,h2,translatt=translatt)
print(comm_4_2)
print('the result of [h4,h3]:')
comm_4_3 = fhc.commutator_translation(h4,h3,translatt=translatt)
print(comm_4_3)

the result of [h4,h2]:
<zero op>
the result of [h4,h3]:
(n_{(0,), up}) @ ((-1.0) Bn_(0,)) + (n_{(1,), up}) @ ((-1.0) Bn_(1,)) + (n_{(0,), dn}) @ ((-1.0) Bn_(0,)) + (n_{(1,), dn}) @ ((-1.0) Bn_(1,))


In [4]:
print('the result of [h3,[h4,h3]]:')
comm_3_4_3 = fhc.simplify(fhc.commutator_translation(h3,comm_4_3,translatt=translatt))
print(comm_3_4_3)
print('the result of [h4,[h4,h3]]:')
comm_4_4_3 = fhc.simplify(fhc.commutator_translation(h4,comm_4_3,translatt=translatt))
print(comm_4_4_3)

the result of [h3,[h4,h3]]:
(-1.0) (n_{(0,), up} + n_{(0,), dn}) @ (Ba_(0,)) + (-1.0) (n_{(1,), up} + n_{(1,), dn}) @ (Ba_(1,))
the result of [h4,[h4,h3]]:
(-2.0) n_{(0,), up} + (-2.0) n_{(1,), up} + (-2.0) n_{(0,), dn} + (-2.0) n_{(1,), dn} + (-4.0) (n_{(0,), up}) @ (n_{(0,), dn}) + (-4.0) (n_{(1,), up}) @ (n_{(1,), dn})


# Get the error bound of commutator

In [5]:
eb_3_4_3 = comm_3_4_3.norm_bound()
eb_4_4_3 = fhc.simplify(comm_4_4_3).norm_bound()
print(eb_3_4_3)
print(eb_4_4_3)

9.33765687335591
16.0


Theoretically, $[h_j,[h4,h3]]=0$, where $j=0,1,2$

Here we test the commutator table to varify the result

In [3]:
comm_tab = fhc.NestedCommutatorTable(hlist, 3, translatt)
tab2 = comm_tab.table(2)
print("tab2[1][4][3]:", tab2[1][4][3])
print("tab2[1][4][3].norm_bound():", tab2[1][4][3].norm_bound())

print("tab2[3][4][3]:", tab2[3][4][3])
print("tab2[3][4][3].norm_bound():", tab2[3][4][3].norm_bound())

tab2[1][4][3]: (-1.0) (g_{(-1,), (0,), up}) @ (Bn_(-1,)) + (g_{(-1,), (0,), up}) @ (Bn_(0,)) + (-1.0) (g_{(-1,), (0,), dn}) @ (Bn_(-1,)) + (g_{(-1,), (0,), dn}) @ (Bn_(0,))
tab2[1][4][3].norm_bound(): 9.337656873355897
tab2[3][4][3]: (-1.0) (n_{(0,), up} + n_{(0,), dn}) @ (Ba_(0,)) + (-1.0) (n_{(1,), up} + n_{(1,), dn}) @ (Ba_(1,))
tab2[3][4][3].norm_bound(): 9.33765687335591


In [7]:
print(tab2[3][4][3].terms[0])
print(comm_3_4_3.terms[0])
tab2[3][4][3].terms[0].proportional(comm_3_4_3.terms[0])

(-1.0) (n_{(0,), up} + n_{(0,), dn}) @ (Ba_(0,))
(-1.0) (n_{(0,), up} + n_{(0,), dn}) @ (Ba_(0,))


True

In [8]:
[[op1,op2] for op1, op2 in zip(tab2[3][4][3].terms[0].ops, comm_3_4_3.terms[0].ops)]

[[<fh_comm.hamiltonian_ops.SumOp at 0x1fcdbf16c50>,

Then we compute the err_bound

In [4]:
print("bound specialized for second-order Suzuki method:")
comm_bound_terms = fhc.commutator_bound_strang(len(hlist))

def get_num_int(idx):
    num_int=0
    for i in idx:
        if i==2:
            num_int+=1
        elif i==3:
            num_int+=3
        elif i==4:
            num_int+=8
    return num_int

# sort by number of interaction terms
err_bound = 19 * [0]
for term in comm_bound_terms:
    #print(term)
    # number of occurrences of interaction term in current nested commutator
    num_int = get_num_int(term.commidx)
    err_bound[num_int] += term.weight * tab2[term.commidx[0]][term.commidx[1]][term.commidx[2]].norm_bound()
# the entries still need to be divided by 2 to obtain the error per lattice site
print("err_bound:", err_bound)

bound specialized for second-order Suzuki method:


ValueError: `A` must not be empty.

In [7]:
print(tab2[0][1][4])

(h_{(-1,), (1,), up}) @ (Ba_(-1,)) + (-1.0) (h_{(-1,), (1,), up}) @ (Ba_(1,)) + (-1.0) (h_{(0,), (2,), up}) @ (Ba_(0,)) + (h_{(0,), (2,), up}) @ (Ba_(2,)) + (h_{(-1,), (1,), dn}) @ (Ba_(-1,)) + (-1.0) (h_{(-1,), (1,), dn}) @ (Ba_(1,)) + (-1.0) (h_{(0,), (2,), dn}) @ (Ba_(0,)) + (h_{(0,), (2,), dn}) @ (Ba_(2,))


In [15]:
temp = fhc.commutator_translation(h1,h4,translatt=translatt)
print(temp)

(g_{(-1,), (0,), up}) @ (Ba_(-1,)) + (-1.0) (g_{(-1,), (0,), up}) @ (Ba_(0,)) + (g_{(-1,), (0,), dn}) @ (Ba_(-1,)) + (-1.0) (g_{(-1,), (0,), dn}) @ (Ba_(0,))


In [18]:
print(fhc.commutator_translation(h0,temp,translatt=translatt))

(h_{(-1,), (1,), up}) @ (Ba_(-1,)) + (-1.0) (h_{(-1,), (1,), up}) @ (Ba_(0,)) + (-1.0) (h_{(0,), (2,), up}) @ (Ba_(1,)) + (h_{(0,), (2,), up}) @ (Ba_(2,)) + (h_{(-1,), (1,), dn}) @ (Ba_(-1,)) + (-1.0) (h_{(-1,), (1,), dn}) @ (Ba_(0,)) + (-1.0) (h_{(0,), (2,), dn}) @ (Ba_(1,)) + (h_{(0,), (2,), dn}) @ (Ba_(2,))


In [2]:
# Add the spin-up and bosonic together
temp_term1 = fhc.ProductOp([fhc.HoppingOp((-1,),(1,),0,1,1,4),fhc.BosonAddOp((-1,),1,4)],1,1,4)
temp_term2 = fhc.ProductOp([fhc.HoppingOp((-1,),(1,),0,1,1,4),fhc.BosonAddOp((0,),1,4)],-1,1,4)
temp_term3 = fhc.ProductOp([fhc.HoppingOp((0,),(2,),0,1,1,4),fhc.BosonAddOp((1,),1,4)],-1,1,4)
temp_term4 = fhc.ProductOp([fhc.HoppingOp((0,),(2,),0,1,1,4),fhc.BosonAddOp((2,),1,4)],1,1,4)
temp_sum1 = fhc.SumOp([temp_term1,temp_term2,temp_term3,temp_term4])
print(temp_sum1.norm_bound())

run the function
9.337656873355929


In [3]:
# Add the spin-down and bosonic together
temp_term5 = fhc.ProductOp([fhc.HoppingOp((-1,),(1,),1,1,1,4),fhc.BosonAddOp((-1,),1,4)],1,1,4)
temp_term6 = fhc.ProductOp([fhc.HoppingOp((-1,),(1,),1,1,1,4),fhc.BosonAddOp((0,),1,4)],-1,1,4)
temp_term7 = fhc.ProductOp([fhc.HoppingOp((0,),(2,),1,1,1,4),fhc.BosonAddOp((1,),1,4)],-1,1,4)
temp_term8 = fhc.ProductOp([fhc.HoppingOp((0,),(2,),1,1,1,4),fhc.BosonAddOp((2,),1,4)],1,1,4)
temp_sum2 = fhc.SumOp([temp_term5,temp_term6,temp_term7,temp_term8])
print(temp_sum2.norm_bound())

run the function
9.337656873355929


In [5]:
temp_sum = fhc.SumOp([temp_sum1,temp_sum2])
print(temp_sum)

(h_{(-1,), (1,), up}) @ (Ba_(-1,)) + (-1) (h_{(-1,), (1,), up}) @ (Ba_(0,)) + (-1) (h_{(0,), (2,), up}) @ (Ba_(1,)) + (h_{(0,), (2,), up}) @ (Ba_(2,)) + (h_{(-1,), (1,), dn}) @ (Ba_(-1,)) + (-1) (h_{(-1,), (1,), dn}) @ (Ba_(0,)) + (-1) (h_{(0,), (2,), dn}) @ (Ba_(1,)) + (h_{(0,), (2,), dn}) @ (Ba_(2,))


In [6]:
print(temp_sum.as_field_operator().as_compact_matrix())

  (4, 1025)	1.0
  (4, 1040)	-1.0
  (5, 1024)	1.0
  (5, 1026)	1.4142135623730951
  (5, 1041)	-1.0
  (6, 1025)	1.4142135623730951
  (6, 1027)	1.7320508075688772
  (6, 1042)	-1.0
  (7, 1026)	1.7320508075688772
  (7, 1043)	-1.0
  (8, 2049)	1.0
  (8, 2064)	-1.0
  (9, 2048)	1.0
  (9, 2050)	1.4142135623730951
  (9, 2065)	-1.0
  (10, 2049)	1.4142135623730951
  (10, 2051)	1.7320508075688772
  (10, 2066)	-1.0
  (11, 2050)	1.7320508075688772
  (11, 2067)	-1.0
  (12, 1033)	-1.0
  (12, 1048)	1.0
  (12, 2053)	1.0
  (12, 2068)	-1.0
  (13, 1032)	-1.0
  :	:
  (65522, 64503)	1.7320508075688772
  (65523, 63467)	1.7320508075688772
  (65523, 63482)	-1.7320508075688772
  (65523, 64487)	-1.7320508075688772
  (65523, 64502)	1.7320508075688772
  (65524, 63468)	1.7320508075688772
  (65524, 63485)	-1.0
  (65525, 63469)	1.7320508075688772
  (65525, 63484)	-1.0
  (65525, 63486)	-1.4142135623730951
  (65526, 63470)	1.7320508075688772
  (65526, 63485)	-1.4142135623730951
  (65526, 63487)	-1.7320508075688772
  (65527

In [26]:
print(temp_sum.norm_bound())

run the function
nan


In [27]:
norm(temp_sum.as_field_operator().as_compact_matrix(),ord=2)

ValueError: `A` must not be empty.